### Basic zip archive data access and usage
For simplicity, the following example loads only 200 sweeps directly from the zipfile into pandas DataFrame objects. These operations can be significantly accelerated using dask instead (see `dask_processing.ipynb`).

In order to run this yourself, you'll need to download a zip file archive, and adjust `data_path` accordingly.

In [1]:
import sea_ingest
from labbench import stopwatch

data_path = "data/2023-09-25_GMM.zip"
with stopwatch("ziparchive read"):
    dfs = sea_ingest.read_seamf_zipfile(
        data_path, allow=200, tz="America/Denver", localize=True
    )

 INFO   2023-10-12 14:05:29.538 • labbench: ziparchive read 2.471 s elapsed


### Returned dictionary structure
The data are returned as a dictionary of `pd.DataFrame`, named by data product or metadata type.

### DataFrame structure
The data products are arranged as tables.
* The trace axis (time elapsed, FFT bin frequency, etc.) is given by the `column` attribute
* The trace index (timestamp, RF center frequency, and any trace specificiations like the detector) are arranged as levels of a multilevel index.

Some advantages to arranging the table this way:
* All data values (below, `dfs['pfp'].values`) are the same kind of quantity, in this case dBm/10 MHz. (TODO: attach units with pint :))
    - This means that operations like `10**(dfs['pfp']/10)` do not apply to the index
* We can use _any_ of the indexing metadata fields to query subsets of the data quickly

In [2]:
dfs["pfp"]

Frame time elapsed (s)                                                     0.000000  \
datetime                         frequency    capture_statistic detector              
2023-09-17 12:00:39.035000-06:00 3.545000e+09 min               rms      -100.00000   
                                              max               rms       -98.56250   
                                              mean              rms       -99.25000   
                                              min               peak      -92.93750   
                                              max               peak      -88.37500   
...                                                                             ...   
2023-09-19 14:59:26.463000-06:00 3.705000e+09 max               rms       -60.25000   
                                              mean              rms       -62.18750   
                                              min               peak      -50.15625   
                                              max               peak      -42.65625   
                                              mean              peak      -45.84375   

Frame time elapsed (s)                                                    0.000018  \
datetime                         frequency    capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 3.545000e+09 min               rms      -99.75000   
                                              max               rms      -98.68750   
                                              mean              rms      -99.18750   
                                              min               peak     -92.93750   
                                              max               peak     -89.12500   
...                                                                            ...   
2023-09-19 14:59:26.463000-06:00 3.705000e+09 max               rms      -60.28125   
                                              mean              rms      -62.12500   
                                              min               peak     -51.84375   
                                              max               peak     -42.15625   
                                              mean              peak     -45.75000   

Frame time elapsed (s)                                                    0.000036  \
datetime                         frequency    capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 3.545000e+09 min               rms      -99.75000   
                                              max               rms      -98.56250   
                                              mean              rms      -99.25000   
                                              min               peak     -93.25000   
                                              max               peak     -87.93750   
...                                                                            ...   
2023-09-19 14:59:26.463000-06:00 3.705000e+09 max               rms      -60.09375   
                                              mean              rms      -62.21875   
                                              min               peak     -51.00000   
                                              max               peak     -41.65625   
                                              mean              peak     -45.93750   

Frame time elapsed (s)                                                    0.000054  \
datetime                         frequency    capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 3.545000e+09 min               rms      -99.81250   
                                              max               rms      -98.62500   
                                              mean              rms      -99.25000   
                                              min               peak     -92.81250   
                                              max               peak     -88.37500   
...                         

### Quick indexing tutorial

You can access each index level using the index value. One way is with the `.loc` accessor, specifying `axis=0` to indicate that all slices are applied to the index (otherwise the 2nd field applies to columns). For example:

In [3]:
dfs["pfp"].loc(axis=0)[:"2023-09-17 12:00", 3.555e9, "max", "rms"]

,,,Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,frequency,capture_statistic,detector,,,,,,,,,,,,,,,,,,,,,
2023-09-17 12:00:39.035000-06:00,3.555000e+09,max,rms,-98.7500,-98.7500,-98.8125,-98.9375,-98.9375,-98.750,-98.7500,-98.9375,-98.9375,-98.8750,...,-94.8125,-98.75,-98.6250,-98.5625,-98.6875,-98.3125,-98.75,-98.6875,-98.75,-98.50
2023-09-17 12:00:54.372000-06:00,3.555000e+09,max,rms,-74.0625,-73.9375,-75.2500,-77.3750,-77.1875,-77.625,-77.0625,-76.7500,-77.1875,-77.1875,...,-75.5625,-76.50,-75.5625,-76.0000,-74.6875,-74.3750,-74.25,-73.0625,-73.75,-73.25


In many cases, we'd like to index a single value in a given level, especially for categorical data like the string-referred `capture_statistic` and `detector` fields. For this, pandas provides the `.xs` accessor.

In [4]:
dfs["pfp"].xs(key=3.555e9, level="frequency")

Frame time elapsed (s)                                       0.000000  \
datetime                         capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 min               rms       -99.9375   
                                 max               rms       -98.7500   
                                 mean              rms       -99.3125   
                                 min               peak      -93.3125   
                                 max               peak      -87.1250   
...                                                               ...   
2023-09-19 14:59:15.984000-06:00 max               rms       -98.9375   
                                 mean              rms       -99.7500   
                                 min               peak      -93.6250   
                                 max               peak      -89.1250   
                                 mean              peak      -91.5625   

Frame time elapsed (s)                                       0.000018  \
datetime                         capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 min               rms      -100.0000   
                                 max               rms       -98.7500   
                                 mean              rms       -99.3125   
                                 min               peak      -93.0625   
                                 max               peak      -88.0625   
...                                                               ...   
2023-09-19 14:59:15.984000-06:00 max               rms       -99.1875   
                                 mean              rms       -99.7500   
                                 min               peak      -93.3750   
                                 max               peak      -89.0000   
                                 mean              peak      -91.5625   

Frame time elapsed (s)                                       0.000036  \
datetime                         capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 min               rms      -100.1875   
                                 max               rms       -98.8125   
                                 mean              rms       -99.4375   
                                 min               peak      -93.3125   
                                 max               peak      -88.9375   
...                                                               ...   
2023-09-19 14:59:15.984000-06:00 max               rms       -99.2500   
                                 mean              rms       -99.7500   
                                 min               peak      -94.0625   
                                 max               peak      -89.1875   
                                 mean              peak      -91.6875   

Frame time elapsed (s)                                       0.000054  \
datetime                         capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 min               rms      -100.1250   
                                 max               rms       -98.9375   
                                 mean              rms       -99.5000   
                                 min               peak      -93.2500   
                                 max               peak      -89.4375   
...                                                               ...   
2023-09-19 14:59:15.984000-06:00 max               rms       -99.3125   
                                 mean              rms       -99.8125   
                                 min               peak      -93.3750   
                                 max               peak      -89.1875   
                                 mean              peak      -91.5000   

Frame time elapsed (s)                                       0.000071  \
datetime                         capture_statistic detector             
2023-09-17 12:00:39.035000-06:00 min               rms

For flexible queries to single index values of a specified data product and multiple index levels at a time, you can also use `seamf.trace`. In the following example, notice that this drops the selected levels from the index. 

In [6]:
sea_ingest.trace(
    dfs, "pfp", frequency=3.555e9, capture_statistic="max", detector="rms"
).loc[:"2023-09-20 20:55"]

Frame time elapsed (s),0.000000,0.000018,0.000036,0.000054,0.000071,0.000089,0.000107,0.000125,0.000143,0.000161,...,0.009821,0.009839,0.009857,0.009875,0.009893,0.009911,0.009929,0.009946,0.009964,0.009982
datetime,,,,,,,,,,,,,,,,,,,,,
2023-09-17 12:00:39.035000-06:00,-98.7500,-98.7500,-98.8125,-98.9375,-98.9375,-98.7500,-98.7500,-98.9375,-98.9375,-98.8750,...,-94.8125,-98.7500,-98.6250,-98.5625,-98.6875,-98.3125,-98.7500,-98.6875,-98.7500,-98.5000
2023-09-17 12:00:54.372000-06:00,-74.0625,-73.9375,-75.2500,-77.3750,-77.1875,-77.6250,-77.0625,-76.7500,-77.1875,-77.1875,...,-75.5625,-76.5000,-75.5625,-76.0000,-74.6875,-74.3750,-74.2500,-73.0625,-73.7500,-73.2500
2023-09-17 12:01:04.815000-06:00,-74.1875,-74.8125,-74.2500,-74.1875,-73.1250,-75.0625,-76.2500,-78.5625,-79.2500,-79.3750,...,-76.5000,-79.0000,-79.6875,-79.5000,-79.4375,-79.5625,-79.3750,-78.6875,-75.0625,-75.0625
2023-09-17 12:01:20.240000-06:00,-75.5625,-77.1875,-78.3125,-78.1250,-79.0625,-78.6875,-82.0000,-81.9375,-81.0625,-76.0000,...,-79.1875,-80.5625,-80.4375,-80.1875,-79.7500,-79.5625,-79.9375,-74.9375,-75.1250,-74.6875
2023-09-17 12:01:35.523000-06:00,-75.6875,-75.3125,-74.6875,-75.0000,-74.8750,-75.3125,-75.4375,-75.1250,-77.9375,-81.3125,...,-98.6875,-99.0000,-98.8750,-99.0000,-99.0000,-98.9375,-98.8750,-99.0625,-99.0625,-99.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-19 14:58:08.032000-06:00,-79.6875,-79.5625,-79.8750,-79.5000,-79.6250,-78.9375,-75.3125,-75.0625,-74.9375,-75.0625,...,-74.6250,-74.5000,-74.7500,-74.6875,-73.6875,-74.5625,-74.3750,-74.3750,-77.9375,-78.8125
2023-09-19 14:58:23.241000-06:00,-74.1250,-74.5000,-73.5625,-73.4375,-72.6875,-73.0625,-73.0625,-73.4375,-73.6875,-73.6875,...,-72.8125,-71.3750,-72.1250,-72.1250,-72.1875,-73.3750,-74.3750,-74.7500,-72.9375,-74.2500
2023-09-19 14:58:38.815000-06:00,-73.9375,-74.5000,-72.5625,-73.5000,-74.3125,-74.9375,-74.3125,-75.0625,-74.1250,-76.3750,...,-72.9375,-72.3750,-72.3125,-74.8750,-75.3750,-75.4375,-75.3750,-74.8125,-75.8750,-76.0625
